<a href="https://colab.research.google.com/github/Abubakar0011/Generative-AI-Finetuning/blob/main/tiny_llama_fine_tuning_on_Bloomberg_financial_news_Abubakar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style economist.",
    },
    {"role": "user", "content": "what if the gdp of any country droped to 2?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0


<|system|>
You are a friendly chatbot who always responds in the style economist.</s>
<|user|>
what if the gdp of any country droped to 2?</s>
<|assistant|>
In that case, the country's economy would be in a state of recession, with a negative GDP figure. The country's economy would be in a state of decline, meaning its output of goods and services would be less than what it was in the previous year. The recession would typically involve falling demand for goods and services, leading to a decline in employment and income, and may also result in lower levels of investment and savings. A country's economy would likely be in a state of stagnation, with no significant improvement expected in the near future.


# Fine Tuning using Lora Teacnique

In [ ]:
!pip install peft transformers bitsandbytes datasets accelerate loralib --upgrade

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 7.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
import torch
from transformers import TrainingArguments
from trl import SFTTrainer

In [ ]:
# Load the dataset
df = load_dataset("danidanou/Bloomberg_Financial_News")

subset_dataset = df["train"].select(range(5000))

# Split the subset into train and validation sets
train_val_split = subset_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_val_split["train"]
eval_dataset = train_val_split["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.00k [00:00<?, ?B/s]

bloomberg_financial_data.parquet.gzip:   0%|          | 0.00/482M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/446762 [00:00<?, ? examples/s]

In [ ]:
# Define the model ID
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj"],
)

In [ ]:
# Load the model and tokenizer with 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# Prepare the model for LoRA training
model = prepare_model_for_kbit_training(model)

# Wrap the model with the LoRA configuration
model = PeftModel(model, lora_config)

In [ ]:
# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(
        examples["Article"], padding="max_length", truncation=True, max_length=512
    )

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)


training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,  # Keep only the last 2 model checkpoints
)

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Create and train the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    args=training_args,
)

<ipython-input-9-16de5097197f>:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,1.997400,No log
2,1.933500,No log


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/tokenizer.model',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(results)

{'eval_runtime': 53.2854, 'eval_samples_per_second': 9.383, 'eval_steps_per_second': 1.182, 'epoch': 2.992}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/MyFineTunedModel"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to: {save_path}")

Mounted at /content/drive
Model and tokenizer saved to: /content/drive/MyDrive/MyFineTunedModel


# After Fine Tuning

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer from Google Drive
model_path = "/content/drive/MyDrive/MyFineTunedModel"

model_ft = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer_ft = AutoTokenizer.from_pretrained(model_path)

print("Model and tokenizer loaded from Google Drive.")

Model and tokenizer loaded from Google Drive.


In [ ]:
# Create the pipeline with the fine-tuned model
pipe_ft = pipeline(
    "text-generation",
    model=model_ft,
    tokenizer=tokenizer_ft,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Use the pipeline for inference
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style economist.",
    },
    {"role": "user", "content": "what if the gdp of any country droped to 2?"},
]
prompt = pipe_ft.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

outputs = pipe_ft(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Device set to use cuda:0


<|system|>
You are a friendly chatbot who always responds in the style economist.</s>
<|user|>
what if the gdp of any country droped to 2?</s>
<|assistant|>
If the GDP (Gross Domestic Product) of any country drops to 2, it means that the economy of that country has contracted. In the context of economics, a drop in GDP means that the economy has shrunk, and it is considered a significant economic contraction. In short, if the GDP of a country drops to 2, it is considered a severe economic downturn, indicating that the country's economy has experienced significant challenges and difficulties.
